In [2]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (5,365 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [77]:
import cv2
import pytesseract
import os
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import warnings
warnings.filterwarnings("ignore")
from PIL import Image
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [4]:
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sentences = []

    for sentence in article:
        #print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

In [5]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [6]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

In [56]:
def generate_summary(file_name, top_n=5):
    nltk.download("stopwords")
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank_numpy(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    #print("Indexes of top ranked_sentence order are ", len(ranked_sentence))    
    if(len(ranked_sentence)>10):
      top_n=10
    else:
      top_n=len(ranked_sentence)
    for i in range(top_n):
      summarize_text.append(" ".join(list(set(ranked_sentence[i][1]))))

    # Step 5 - Offcourse, output the summarize texr
    return ". ".join(summarize_text)

In [ ]:
input_data=[]
output_data=[]
for i in range(1,81):
  path='/content/valid/'
  if(i<27):
    path=path+str(i)+'.png'
  elif(i>=27 and i<=76):
    path=path+str(i)+'.jpg'
  else:
    path=path+str(i)+'.jpeg'
  img = Image.open(path)
  out_below = pytesseract.image_to_string(img)
  out_below=out_below.split('\n')
  out_below=' '.join(out_below)
  with open('exp.txt', 'w') as writefile:
    writefile.write(out_below)
  summary=generate_summary( "exp.txt")
  if(len(summary)!=0):
    input_data.append(summary)
    output_data.append(1)

In [65]:
print(len(input_data))
print(len(output_data))

76
76


In [ ]:
data_in=[]
data_out=[]
for i in range(1,69):
  path='/content/invalid/'
  if(i<28):
    path=path+str(i)+'.png'
  elif(i>=28 and i<=67):
    path=path+str(i)+'.jpg'
  else:
    path=path+str(i)+'.jpeg'
  img = Image.open(path)
  out_below = pytesseract.image_to_string(img)
  out_below=out_below.split('\n')
  out_below=' '.join(out_below)
  with open('exp.txt', 'w') as writefile:
    writefile.write(out_below)
  summary=generate_summary( "exp.txt")
  if(len(summary)!=0):
    data_in.append(summary)
    data_out.append(0)

In [62]:
print(type(data_in[0]))
print(len(data_out))

<class 'str'>
54


In [66]:
data_1=input_data
data_2=data_in

In [67]:
data = open('corpus').read()

In [68]:
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(0)
    texts.append(" ".join(content[1:]))
    if(i==99):
      break

In [70]:
print(len(texts))
print(len(labels))

100
100


In [71]:
data_in=data_in+texts
data_out=data_out+labels

In [72]:
print(len(data_in))
print(len(data_out))

154
154


In [73]:
input_data=input_data+data_in
output_data=output_data+data_out

In [74]:
print(len(input_data))
print(len(output_data))

230
230


In [79]:
data = pd.DataFrame()
data['text'] = input_data
data['label'] = output_data

In [80]:
data.head(5)

,text,label
0,05-Jun-1992 RAMNARESH VISHWA VINAYAK 310291 o...,1
1,"CENTER 600008, DISCHARGE NALAM& ~ MEDICARE [E...",1
2,MUKHOPADHYAY was on by: (Consultant seen refe...,1
3,year 08/01/14 by: 8/1/14 Record weakness Admi...,1
4,All has with information the been patient of ...,1


In [110]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(data['text'], data['label'], test_size = 0.25)
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(y_train)
valid_y = encoder.fit_transform(y_test)

In [113]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(data['text'])
x_train_tf_idf =  tfidf_vect.transform(x_train)
x_test_tf_idf =  tfidf_vect.transform(x_test)

In [114]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(x_train_tf_idf.toarray(), y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [115]:
y_pred = classifier.predict(x_test_tf_idf.toarray())

In [117]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)

In [118]:
print('confusion matrix :\n',cm)

confusion matrix :
 [[43  0]
 [ 3 12]]


In [119]:
from sklearn.metrics import accuracy_score
print ("Accuracy : ", accuracy_score(y_test, y_pred))

Accuracy :  0.9482758620689655
